In [ ]:
OPENAI_API_KEY = "YOUR_OPENAI_API_KEY_HERE"

In [6]:
from intent_kit import IntentGraphBuilder, handler, llm_classifier

# LLM configuration (remove if not using LLM-powered features)
LLM_CONFIG = {
    "provider": "openai",
    "api_key": OPENAI_API_KEY,
    "model": "gpt-4.1-mini",
}


def _calculate_handler(operation: str, a: float, b: float) -> str:
    """Handle calculation with proper operator mapping."""
    operation_map = {
        "plus": "+",
        "add": "+",
        "minus": "-",
        "subtract": "-",
        "times": "*",
        "multiply": "*",
        "multiplied": "*",
        "divided": "/",
        "divide": "/",
        "over": "/",
    }
    math_op = operation_map.get(operation.lower(), operation)
    try:
        result = eval(f"{a} {math_op} {b}")
        return f"{a} {operation} {b} = {result}"
    except (SyntaxError, ZeroDivisionError) as e:
        return f"Error: Cannot calculate {a} {operation} {b} - {str(e)}"


def create_intent_graph():
    """Create and configure the intent graph."""
    handlers = [
        handler(
            name="greet",
            description="Greet the user",
            handler_func=lambda name, **kwargs: f"Hello {name}!",
            param_schema={"name": str},
            llm_config=LLM_CONFIG,
        ),
        handler(
            name="calculate",
            description="Perform a calculation",
            handler_func=lambda operation, a, b, **kwargs: _calculate_handler(
                operation, a, b
            ),
            param_schema={"operation": str, "a": float, "b": float},
            llm_config=LLM_CONFIG,
        ),
        handler(
            name="weather",
            description="Get weather information",
            handler_func=lambda location, **kwargs: f"Weather in {location}: 72°F, Sunny (simulated)",
            param_schema={"location": str},
            llm_config=LLM_CONFIG,
        ),
        handler(
            name="help",
            description="Get help",
            handler_func=lambda **kwargs: "I can help with greetings, calculations, and weather!",
            param_schema={},
            llm_config=LLM_CONFIG,
        ),
    ]
    classifier = llm_classifier(
        name="root",
        children=handlers,
        llm_config=LLM_CONFIG,
        description="Main intent classifier",
    )
    return IntentGraphBuilder().root(classifier).build()

In [7]:
# Test the graph
from intent_kit.context import IntentContext

graph = create_intent_graph()
context = IntentContext(session_id="simple_demo")

test_inputs = [
    "Hello, my name is Alice",
    "What's 15 plus 7?",
    "Weather in San Francisco",
    "Help me",
    "Multiply 8 and 3",
]

for user_input in test_inputs:
    print(f"\nInput: {user_input}")
    result = graph.route(user_input, context=context)
    if result.success:
        print(f"Intent: {result.node_name}")
        print(f"Output: {result.output}")
    else:
        print(f"Error: {result.error}")

[INFO] [intent_kit.graph.intent_graph] Added root node: root
[INFO] [intent_kit.graph.intent_graph] Validating graph structure...
[DEBUG] [intent_kit.graph.validation] Validating node types...
[INFO] [intent_kit.graph.validation] Node type validation passed ✓
[DEBUG] [intent_kit.graph.validation] Validating splitter-to-classifier routing constraints...
[INFO] [intent_kit.graph.validation] Splitter routing validation passed ✓
[DEBUG] [intent_kit.graph.validation] Validating graph structure...
[DEBUG] [intent_kit.graph.validation] Validating splitter-to-classifier routing constraints...
[INFO] [intent_kit.graph.validation] Splitter routing validation passed ✓
[INFO] [intent_kit.graph.validation] Graph validation complete: 5 total nodes, routing valid: True, cycles: False
[INFO] [intent_kit.graph.intent_graph] Graph validation completed successfully
[INFO] [intent_kit.graph.intent_graph] Graph validation passed after adding root node

Input: Hello, my name is Alice
[DEBUG] [root] Classifi